# Caracterização das mudanças em bug reports

## Base
A base de dados é composta por 690.817 bug reports.

## Filtro da base
*   Product: todos
*   Status: RESOLVED
*   Data de criação do bug report: entre 01/01/2013 e 01/01/2022


## Importações

In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

plt.style.use('ggplot')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

df_bugs = pd.read_json('processed_cf_notcf_counting_final.json')
df_fields = pd.read_json('processed_verify_fields_final.json')

df_fields = df_fields.drop(columns=['bug_id'])
df_bugs = df_bugs.drop(columns=['bug_id'])

## 1. Total de campos únicos que registraram ao menos uma modificação na base de dados

In [2]:
total_fields = len(df_fields)
total_fields

617

## 2. Total de campos *custom field* e total de campos não *custom field*

In [3]:
total_nom_custom_fields = df_fields.query('~ (name.str.startswith("cf_"))', engine='python').size
total_custom_fields = df_fields.query('name.str.startswith("cf_")', engine='python').size

print(f'Total de custom fields: {total_custom_fields}')
print(f'Total de não custom fields: {total_nom_custom_fields}')
print(f'Porcentagem de custom fields: {(total_custom_fields / total_fields) * 100: ,.2f}%')
print(f'Porcentagem de não custom fields: {(total_nom_custom_fields / total_fields) * 100: ,.2f}%')

Total de custom fields: 578
Total de não custom fields: 39
Porcentagem de custom fields:  93.68%
Porcentagem de não custom fields:  6.32%


## 3. Estatísticas do total de modificações por bug report para os campos *custom field* e para os campos não *custom field*

In [4]:
df_bugs.columns = ['custom_field', 'not_custom_field']
df_bugs.describe()

,custom_field,not_custom_field
count,690817.00,690817.00
mean,2.04,12.24
std,2.42,15.20
min,1.00,2.00
25%,1.00,5.00
50%,1.00,8.00
75%,2.00,14.00
max,94.00,1793.00


## 4. Considerando todas as modificações sejam elas *custom field* ou não

In [5]:
all_bugs = df_bugs['custom_field'] + df_bugs['not_custom_field']
all_bugs.describe()

count   690817.00
mean        14.29
std         16.25
min          3.00
25%          6.00
50%         10.00
75%         17.00
max       1796.00
dtype: float64

## 5. Campos *custom field* que não são específicos de versões de produto

### 5.1 Total de campos

In [6]:
df_cf_not_version = df_fields.query('(~ (name.str.startswith("cf_status") or name.str.startswith("cf_tracking"))) and name.str.startswith("cf_")', engine='python').reset_index(drop=True)
len(df_cf_not_version)

34

### 5.2 Nome dos campos

In [7]:
df_cf_not_version

,name
0,cf_last_resolved
1,cf_crash_signature
2,cf_colo_site
3,cf_blocking_basecamp
4,cf_blocking_b2g
5,cf_rank
6,cf_backlog
7,cf_blocking_fennec
8,cf_qa_whiteboard
9,cf_performance


## 6. Os 10 campos *custom field* com mais presença na base (campos de versão foram filtrados)

In [8]:
DATABASE_SIZE = 690817

with open('processed_bugs_cfs_presence.json') as input:
  cf_fields_presence_dict = json.load(input)

cf_fields_presence_list = []

for key in cf_fields_presence_dict.keys():
  cf_fields_presence_list.append({
    'custom_field_name': key,
    'percentage': (cf_fields_presence_dict[key]/DATABASE_SIZE) * 100
  })

df_cfs_percentage = pd.DataFrame(cf_fields_presence_list)

df_cfs_percentage_filtered = df_cfs_percentage.query('~ (custom_field_name.str.startswith("cf_status") or custom_field_name.str.startswith("cf_tracking"))', engine='python')
df_cfs_percentage_filtered.sort_values(ascending=False, by='percentage').head(10).reset_index(drop=True)

,custom_field_name,percentage
0,cf_last_resolved,100.00
1,cf_blocking_b2g,2.48
2,cf_qa_whiteboard,1.68
3,cf_has_regression_range,1.61
4,cf_crash_signature,0.96
5,cf_fx_iteration,0.91
6,cf_rank,0.82
7,cf_fx_points,0.81
8,cf_has_str,0.59
9,cf_blocking_fennec,0.57
